In [10]:
import fastf1
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

In [2]:
fastf1.Cache.enable_cache('fastf1_cache')

In [6]:
race=fastf1.get_session(2023,"Monaco",'R')
race.load(weather=True)
weather=race.weather_data
avgtemp=weather['AirTemp'].mean()
rainfall=weather['Rainfall'].sum()
df=pd.DataFrame({
    "Driver":race.laps['Driver'],
    'Position':race.laps['Position'],
    "Stint":race.laps["Stint"],
    "Compound":race.laps["Compound"],
    "LapNumber":race.laps["LapNumber"],
    "avgtemp":avgtemp,
    "rainfall":rainfall
})

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '14', '31', '44', '63', '16', '10', '55', '4', '81', '77', '21', '24', '23', '22', '11', '27', '2', '20', '18']


In [17]:
data=df.copy()
le = LabelEncoder()
data["Driver"] = le.fit_transform(data["Driver"])
data["Compound"] = le.fit_transform(data["Compound"])
print(data.head())
y=data['Compound']
x=data.drop('Compound',axis=1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=420)
        


   Driver  Position  Stint  Compound  LapNumber    avgtemp  rainfall
0      18       1.0    1.0         2        1.0  25.065909        26
1      18       1.0    1.0         2        2.0  25.065909        26
2      18       1.0    1.0         2        3.0  25.065909        26
3      18       1.0    1.0         2        4.0  25.065909        26
4      18       1.0    1.0         2        5.0  25.065909        26


In [18]:
model=xgb.XGBClassifier(
    objective="multi:softprob",  
    num_class=len(le.classes_),  
    learning_rate=0.1,
    max_depth=5
)

model.fit(X_train, y_train)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=5,
              num_parallel_tree=None, ...)

In [19]:
pred = model.predict(X_test)
print("Predicted strategies:", le.inverse_transform(pred))

Predicted strategies: ['HARD' 'MEDIUM' 'HARD' 'HARD' 'INTERMEDIATE' 'HARD' 'INTERMEDIATE'
 'MEDIUM' 'INTERMEDIATE' 'MEDIUM' 'INTERMEDIATE' 'HARD' 'INTERMEDIATE'
 'INTERMEDIATE' 'MEDIUM' 'HARD' 'HARD' 'INTERMEDIATE' 'INTERMEDIATE'
 'SOFT' 'INTERMEDIATE' 'MEDIUM' 'HARD' 'HARD' 'MEDIUM' 'INTERMEDIATE'
 'MEDIUM' 'HARD' 'INTERMEDIATE' 'INTERMEDIATE' 'MEDIUM' 'INTERMEDIATE'
 'WET' 'INTERMEDIATE' 'MEDIUM' 'HARD' 'MEDIUM' 'MEDIUM' 'HARD' 'HARD'
 'INTERMEDIATE' 'HARD' 'INTERMEDIATE' 'MEDIUM' 'HARD' 'INTERMEDIATE'
 'INTERMEDIATE' 'HARD' 'INTERMEDIATE' 'WET' 'HARD' 'HARD' 'INTERMEDIATE'
 'MEDIUM' 'MEDIUM' 'MEDIUM' 'HARD' 'MEDIUM' 'MEDIUM' 'HARD' 'MEDIUM'
 'INTERMEDIATE' 'HARD' 'INTERMEDIATE' 'MEDIUM' 'HARD' 'HARD' 'HARD'
 'MEDIUM' 'MEDIUM' 'HARD' 'INTERMEDIATE' 'MEDIUM' 'WET' 'INTERMEDIATE'
 'HARD' 'HARD' 'MEDIUM' 'HARD' 'HARD' 'HARD' 'MEDIUM' 'INTERMEDIATE'
 'HARD' 'MEDIUM' 'HARD' 'HARD' 'HARD' 'HARD' 'HARD' 'HARD' 'MEDIUM'
 'MEDIUM' 'INTERMEDIATE' 'HARD' 'INTERMEDIATE' 'WET' 'INTERMEDIATE' 'HAR